In [1]:
import os
import subprocess
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import pinecone
from sentence_transformers import SentenceTransformer

# Get environment variables for Pinecone API key, environment, and index name.
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')
dimension = 768

In [2]:
# Set embedding model
EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"

# Load the model stored in models/embedding-model
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_REPO)
model = AutoModel.from_pretrained(EMBEDDING_MODEL_REPO)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
2024-05-14 16:50:27.617208: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-14 16:50:28.237439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 16:50:28.237472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 16:50:28.266181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 

In [3]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
print("Pinecone initialised")

Pinecone initialised


In [5]:
def create_pinecone_collection(PINECONE_INDEX):
    try:
        print(f"Creating 768-dimensional index called '{PINECONE_INDEX}'...")
        # Create the Pinecone index with the specified dimension.
        pinecone.create_index(PINECONE_INDEX, dimension=768)
        print("Success")
    except:
        # index already created, continue
        pass

    print("Checking Pinecone for active indexes...")
    active_indexes = pinecone.list_indexes()
    print("Active indexes:")
    print(active_indexes)
    print(f"Getting description for '{PINECONE_INDEX}'...")
    index_description = pinecone.describe_index(PINECONE_INDEX)
    print("Description:")
    print(index_description)

    print(f"Getting '{PINECONE_INDEX}' as object...")
    pinecone_index = pinecone.Index(PINECONE_INDEX)
    print("Success")

    # Return the Pinecone index object.
    return pinecone_index

In [6]:
collection = create_pinecone_collection(PINECONE_INDEX)

Creating 768-dimensional index called 'cml-index-se-west'...
Checking Pinecone for active indexes...


TypeError: expected string or bytes-like object